In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import keras.layers as L
import keras.models as M
import tensorflow as tf
from sklearn.metrics import *
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

np.random.seed(51)

Using TensorFlow backend.


In [2]:
def create_time_stepped_data(df, time_steps,num_features):
    d = []
    for i in range(time_steps):
        d.append(df.shift(-i).values[:-time_steps].reshape(-1,num_features+1))
    return np.transpose(np.array(d),(1,0,2))

In [3]:
def model_define(input_shape):
    
    inp = L.Input(shape=input_shape)
    
    x = L.LSTM(256, recurrent_dropout=0.1, activation = 'relu',return_sequences=True)(inp)
    x = L.LSTM(256, recurrent_dropout=0.1, activation = 'relu',return_sequences=True)(x)
    x = L.LSTM(256, recurrent_dropout=0.1, activation = 'relu',return_sequences=True)(x)
    
    x = L.Dense(1024)(x)
    x = L.Dropout(0.1)(x)
    x = L.advanced_activations.LeakyReLU(0.3)(x)
    x = L.Dense(1024)(x)
    x = L.Dropout(0.1)(x)
    x = L.advanced_activations.LeakyReLU(0.3)(x)
    x = L.Dense(1024)(x)
    x = L.Dropout(0.1)(x)
    x = L.advanced_activations.LeakyReLU(0.3)(x)
    x = L.Dense(1024)(x)
    x = L.Dropout(0.1)(x)
    x = L.advanced_activations.LeakyReLU(0.3)(x)
    x = L.Dense(1)(x)
    out = L.Dropout(0.1)(x)
    
    model = M.Model(inputs=[inp], outputs=[out])
    model.compile(optimizer='adam', loss=tf.losses.huber_loss, metrics=['mae'])
    
    return model

In [4]:
df = pd.read_csv("../New Data/Data.csv")
l = list(range(81,89))
data = df.iloc[:,l]

In [ ]:
for i in df.columns:
    print(i)

In [5]:
cols = ['food1', 'food2', 'food3', 'food4', 'food5', 'food6', 'food7',
       'food8', 'food9', 'food10', 'food11', 'food12', 'food13', 'food14']

In [ ]:
writer = pd.ExcelWriter('../Results/LSTMresults.xlsx')
split = int(0.7*data.shape[0])
for food in cols:
    d = pd.DataFrame()
    print ("\n\nFood : ", food)
    print ("-------------------------------------------------------------")
    r = create_time_stepped_data(data[food],14,0)
    x = r[:,:-7,:].reshape((-1,7,1))
    y = r[:,7:,:].reshape((-1,7,1))
    model = model_define(x.shape[1:])
    model.summary()
    x_train, x_test = x[:split], x[split:]
    y_train, y_test = y[:split], y[split:]
    print(x_train.shape, x_test.shape)
    model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test,y_test))
    pred = model.predict(x_test,batch_size=32)[:,-1,:].reshape((-1,))
    test = y_test[:,-1,:].reshape((-1,))
    d['Date'] = df.values[split+7:-7,0]
    d['yPred'] = pred
    d['yTest'] = test
    d['mape'] = np.abs(pred - test ) / test * 100
    d.to_excel(writer,sheet_name=food,index=False)
writer.save()